In [1]:
##install pubchem python library
!pip install pubchempy

In [2]:
import requests
from bs4 import BeautifulSoup
import re
import pubchempy as pcp
import pandas as pd

In [3]:
### Function to scrap the data from PubChem
def cid_to_sol(id):
   x="https://pubchem.ncbi.nlm.nih.gov/rest/pug_view/data/compound/%s/xml"
   data=requests.get(x % id)#,timeout=(10,20) )
   html = BeautifulSoup(data.content, "xml")
   solubility = html.find(name='TOCHeading', string='Solubility')
   #print(dtype(solubility))
   if solubility == None:
      return None
   #elif          # or pass
   else:
     solub=solubility.find_next_sibling('Information').find(name='String').string
     return solub

In [ ]:
### Another function to scrap the data based on smiles 
def smiles_to_sol(SMILES):
    try:
        prop = pcp.get_properties(['MolecularWeight'], SMILES, 'smiles')
        x = list(map(lambda x: x["CID"], prop))
        if not x:
            return None
        y = x[0]

        url = "https://pubchem.ncbi.nlm.nih.gov/rest/pug_view/data/compound/%s/xml" % y
        data = requests.get(url)

        if data.status_code != 200:
            return None

        html = BeautifulSoup(data.content, "xml")
        solubility = html.find(name='TOCHeading', string='Solubility')
        if solubility is None:
            return None
        else:
            solub = solubility.find_next_sibling('Information').find(name='String').string
            return solub
    except Exception as e:
        print(f"An error occurred: {e}")
        return None



In [4]:
df = pd.read_csv('pubchem_0k_10k.csv')

In [5]:
print(len(df))

10000


In [35]:
def cid_to_sol(id):
   x="https://pubchem.ncbi.nlm.nih.gov/rest/pug_view/data/compound/%s/xml"
   data=requests.get(x % id)#,timeout=(10,20) )
   html = BeautifulSoup(data.content, "xml")
   solubility = html.find(name='TOCHeading', string='Solubility')

   if solubility == None:
      return None

   else:
     solub=solubility.find_next_sibling('Information').find(name='String').string
     return solub.string
x=cid_to_sol(110635)
print(x)

3.2 [ug/mL] (The mean of the results at pH 7.4)


In [8]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import time
from concurrent.futures import ThreadPoolExecutor

start = time.time()

df = pd.read_csv('pubchem_0k_10k.csv')
#df = pd.read_csv('/content/drive/MyDrive/pubchem_23000k_24000k.csv')
df_list = [df.iloc[i*1000:(i+1)*1000] for i in range(10)]

def cid_to_sol(id):
    url = f"https://pubchem.ncbi.nlm.nih.gov/rest/pug_view/data/compound/{id}/xml"
    try:
        data = requests.get(url)
        if data.status_code == 200:
            html = BeautifulSoup(data.content, "xml")
            solubility = html.find(name='TOCHeading', string='Solubility')

            if solubility is None:
                return None#'Data not found for this compound'
            else:
                solub = solubility.find_next_sibling('Information').find(name='String').string
                return solub
        else:
            return None#f'Error: Server returned status code {data.status_code}'
    except requests.exceptions.RequestException as e:
        return None#f'Error: Request Exception - {e}'

def process_chunk(df_chunk):
    P_sol = []
    for i in range(len(df_chunk)):
        try:
            time.sleep(0.6)  # Sleep for 0.6 seconds
            df_chunk = df_chunk.reset_index(drop=True)
            sol = cid_to_sol(df_chunk.CID[i])
            P_sol.append(sol)
        except AttributeError as e:
            sol = 'No string'
            P_sol.append(sol)
    return P_sol

p_sol_final = []

with ThreadPoolExecutor(max_workers=10) as executor:
    results = executor.map(process_chunk, df_list)
    for result in results:
        p_sol_final.append(result)

df1 = pd.DataFrame(p_sol_final)
joinlist = [x for sublist in p_sol_final for x in sublist]
df1 = pd.DataFrame(joinlist)
df = pd.concat(df_list)
df1 = df1.reset_index(drop=True)
df = df.reset_index(drop=True)
data = pd.concat([df, df1], axis=1, join='inner')
data.to_csv('pubchem_new_0_10k.csv',encoding='utf-8')


end = time.time()
duration = end - start
print(duration)

encoding error : input conversion failed due to input error, bytes 0x90 0x32 0x2C 0x34
encoding error : input conversion failed due to input error, bytes 0x90 0x32 0x2C 0x34
I/O error : encoder error
encoding error : input conversion failed due to input error, bytes 0x8F 0xC3 0x82 0x89
encoding error : input conversion failed due to input error, bytes 0x8F 0xC3 0x82 0x89
I/O error : encoder error
encoding error : input conversion failed due to input error, bytes 0x8E 0xC2 0xB1 0x2D
encoding error : input conversion failed due to input error, bytes 0x8E 0xC2 0xB1 0x2D
I/O error : encoder error
encoding error : input conversion failed due to input error, bytes 0x81 0xC2 0xB5 0x2C
encoding error : input conversion failed due to input error, bytes 0x81 0xC2 0xB5 0x2C
I/O error : encoder error
encoding error : input conversion failed due to input error, bytes 0x9D 0x29 0x20 0x63
encoding error : input conversion failed due to input error, bytes 0x9D 0x29 0x20 0x63
I/O error : encoder error


4446.9308269023895


In [32]:
### Checking the null values in the raw file so that can have which has the solubility values 
final_data=pd.read_csv('pubchem_new_0_10000.csv')
final_data_raw=final_data[['CID','SMILES','0']]
final_data_raw.rename(columns = {'0':'Solubility'}, inplace = True)
final_raw = final_data_raw.loc[final_data_raw['Solubility'].notnull()]
print(len(final_raw))


1


/var/folders/k_/8s01c4dj0gnfx2n284nvm1t80000gn/T/ipykernel_13058/2306803033.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_data_raw.rename(columns = {'0':'Solubility'}, inplace = True)


In [33]:
final_raw

,CID,SMILES,Solubility
2309,2466,CCCCOC1=CC=C(C=C1)CC(=O)NO,>33.5 [ug/mL] (The mean of the results at pH 7.4)
